In [1]:
# Import libraries

import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns


# Pretty display for notebooks

%matplotlib inline


# Allows the use of display() for DataFrames
from IPython.display import display 

# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
train = pd.read_csv("../asset/train.csv")
test = pd.read_csv("../asset/test.csv")
weather = pd.read_csv("../asset/weather.csv")
key = pd.read_csv("../asset/key.csv")
submission_example = pd.read_csv("../asset/sampleSubmission.csv")

# Success - Display the first record
print("Train data : ", train.shape)
print("Test  data : ", test.shape)
print("weather  data : ", weather.shape)
print("key  data : ", key.shape)
print("submission  data : ", submission_example.shape)

import pickle
import awesome_functions as cf

slack_url = pickle.load(open('slackUrl.pickle', 'rb'))
slack_url

# 원본을 유지하기 위해서 카피
df_train = train.copy()
df_weather = weather.copy()
df_key = key.copy()
df_test = test.copy()

Train data :  (4617600, 4)
Test  data :  (526917, 3)
weather  data :  (20517, 20)
key  data :  (45, 2)
submission  data :  (526917, 2)


In [2]:
df_weather.tail()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
20512,16,2014-10-31,53,34,44,M,35,41,21,0,-,-,,M,0.00,29.90,29.99,4.5,03,5.3
20513,17,2014-10-31,59,34,47,M,32,41,18,0,-,-,RA,0.0,0.00,29.72,30.39,9.2,01,9.3
20514,18,2014-10-31,67,49,58,-4,40,50,7,0,0644,1738,,0.0,0.00,29.78,30.28,10.6,36,11.2
20515,19,2014-10-31,45,33,39,-6,24,32,26,0,0624,1646,RA SN,0.1,0.02,29.51,30.24,20.4,34,20.9
20516,20,2014-10-31,68,50,59,M,39,50,6,0,-,-,,0.0,0.00,29.57,30.27,10.5,36,11.2


In [3]:
drop_ls = ["station_nbr","date","sunrise", "sunset", "codesum"]
df_weather_drop = df_weather.drop(drop_ls, axis=1)
df_weather_drop.tail()

,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
20512,53,34,44,M,35,41,21,0,M,0.00,29.90,29.99,4.5,03,5.3
20513,59,34,47,M,32,41,18,0,0.0,0.00,29.72,30.39,9.2,01,9.3
20514,67,49,58,-4,40,50,7,0,0.0,0.00,29.78,30.28,10.6,36,11.2
20515,45,33,39,-6,24,32,26,0,0.1,0.02,29.51,30.24,20.4,34,20.9
20516,68,50,59,M,39,50,6,0,0.0,0.00,29.57,30.27,10.5,36,11.2


In [4]:
cf.isThereNoneData(df_weather_drop)

M Data over 60% : []
T Data over 60% : []
B Data over 60% : []


,Column,Row Count,Missing Data,M Data %,Trace Data,T Data %,Bar Data,B Data %
0,tmax,20517,906,4.42,0,0.00,0,0.0
1,tmin,20517,908,4.43,0,0.00,0,0.0
2,tavg,20517,1469,7.16,0,0.00,0,0.0
3,depart,20517,11511,56.10,0,0.00,0,0.0
4,dewpoint,20517,666,3.25,0,0.00,0,0.0
5,wetbulb,20517,1252,6.10,0,0.00,0,0.0
6,heat,20517,1469,7.16,0,0.00,0,0.0
7,cool,20517,1469,7.16,0,0.00,0,0.0
8,snowfall,20517,7224,35.21,311,1.52,0,0.0
9,preciptotal,20517,860,4.19,2410,11.75,0,0.0


In [5]:
df_weather_drop_nonM = cf.remove_m_row(df_weather_drop)
df_weather_drop_nonM.tail()

원본 데이터프레임 length : 20517
제거하고 싶은 컬럼 : ['tmax', 'tmin', 'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir', 'avgspeed']
제거한 int 컬럼 : ['tmax', 'tmin', 'tavg', 'dewpoint', 'wetbulb', 'heat', 'cool']
제거한 float 컬럼 : ['depart', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir', 'avgspeed']
제거한 후 데이터프레임 length : 8763


,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
20507,75.0,50.0,63.0,-2.0,53.0,57.0,2.0,0.0,0.0,0.00,29.82,29.85,5.3,30.0,6.1
20510,60.0,38.0,49.0,-7.0,31.0,42.0,16.0,0.0,0.0,0.00,28.98,30.36,11.9,1.0,12.5
20511,50.0,32.0,41.0,-2.0,34.0,38.0,24.0,0.0,0.0,0.00,29.65,30.03,0.3,3.0,0.6
20514,67.0,49.0,58.0,-4.0,40.0,50.0,7.0,0.0,0.0,0.00,29.78,30.28,10.6,36.0,11.2
20515,45.0,33.0,39.0,-6.0,24.0,32.0,26.0,0.0,0.1,0.02,29.51,30.24,20.4,34.0,20.9


In [6]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df_weather_drop_nonM.values, i) for i in range(df_weather_drop_nonM.shape[1])]
vif["features"] = df_weather_drop_nonM.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif

,VIF Factor,features
0,1.108916,snowfall
1,1.140053,resultdir
2,1.182985,preciptotal
3,1.377830,depart
4,1.611556,stnpressure
5,1.905672,sealevel
6,8.157570,resultspeed
7,8.834553,avgspeed
8,96.121497,dewpoint
9,313.719123,wetbulb


### tmax, tmin 삭제

In [8]:
drop_ls_1 = ["station_nbr","date","sunrise", "sunset", "codesum", "tmax", "tmin"]
df_weather_drop_1 = df_weather.drop(drop_ls_1, axis=1)
df_weather_drop_1.tail()

,tavg,depart,dewpoint,wetbulb,heat,cool,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
20512,44,M,35,41,21,0,M,0.00,29.90,29.99,4.5,03,5.3
20513,47,M,32,41,18,0,0.0,0.00,29.72,30.39,9.2,01,9.3
20514,58,-4,40,50,7,0,0.0,0.00,29.78,30.28,10.6,36,11.2
20515,39,-6,24,32,26,0,0.1,0.02,29.51,30.24,20.4,34,20.9
20516,59,M,39,50,6,0,0.0,0.00,29.57,30.27,10.5,36,11.2


In [9]:
df_weather_drop_nonM_1 = cf.remove_m_row(df_weather_drop_1)
df_weather_drop_nonM_1.tail()

원본 데이터프레임 length : 20517
제거하고 싶은 컬럼 : ['tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir', 'avgspeed']
제거한 int 컬럼 : ['tavg', 'dewpoint', 'wetbulb', 'heat', 'cool']
제거한 float 컬럼 : ['depart', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir', 'avgspeed']
제거한 후 데이터프레임 length : 8763


,tavg,depart,dewpoint,wetbulb,heat,cool,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
20507,63.0,-2.0,53.0,57.0,2.0,0.0,0.0,0.00,29.82,29.85,5.3,30.0,6.1
20510,49.0,-7.0,31.0,42.0,16.0,0.0,0.0,0.00,28.98,30.36,11.9,1.0,12.5
20511,41.0,-2.0,34.0,38.0,24.0,0.0,0.0,0.00,29.65,30.03,0.3,3.0,0.6
20514,58.0,-4.0,40.0,50.0,7.0,0.0,0.0,0.00,29.78,30.28,10.6,36.0,11.2
20515,39.0,-6.0,24.0,32.0,26.0,0.0,0.1,0.02,29.51,30.24,20.4,34.0,20.9


In [10]:
vif_1 = pd.DataFrame()
vif_1["VIF Factor"] = [variance_inflation_factor(df_weather_drop_nonM_1.values, i) for i in range(df_weather_drop_nonM_1.shape[1])]
vif_1["features"] = df_weather_drop_nonM_1.columns
vif_1 = vif_1.sort_values("VIF Factor").reset_index(drop=True)
vif_1

,VIF Factor,features
0,1.108833,snowfall
1,1.136262,resultdir
2,1.174026,preciptotal
3,1.366198,depart
4,1.595787,stnpressure
5,1.902218,sealevel
6,8.062364,resultspeed
7,8.833752,avgspeed
8,90.967992,dewpoint
9,309.795815,wetbulb


### tavg > sealevel > stnpressure > wetbulb 삭제

In [11]:
drop_ls_2 = ["station_nbr","date","sunrise", "sunset", "codesum", "tavg", "sealevel", "stnpressure", "wetbulb"]

df_weather_drop_2 = df_weather.drop(drop_ls_2, axis=1)
df_weather_drop_2.tail()

,tmax,tmin,depart,dewpoint,heat,cool,snowfall,preciptotal,resultspeed,resultdir,avgspeed
20512,53,34,M,35,21,0,M,0.00,4.5,03,5.3
20513,59,34,M,32,18,0,0.0,0.00,9.2,01,9.3
20514,67,49,-4,40,7,0,0.0,0.00,10.6,36,11.2
20515,45,33,-6,24,26,0,0.1,0.02,20.4,34,20.9
20516,68,50,M,39,6,0,0.0,0.00,10.5,36,11.2


In [12]:
df_weather_drop_nonM_2 = cf.remove_m_row(df_weather_drop_2)
df_weather_drop_nonM_2.tail()

원본 데이터프레임 length : 20517
제거하고 싶은 컬럼 : ['tmax', 'tmin', 'depart', 'dewpoint', 'heat', 'cool', 'snowfall', 'preciptotal', 'resultspeed', 'resultdir', 'avgspeed']
제거한 int 컬럼 : ['tmax', 'tmin', 'dewpoint', 'heat', 'cool']
제거한 float 컬럼 : ['depart', 'snowfall', 'preciptotal', 'resultspeed', 'resultdir', 'avgspeed']
제거한 후 데이터프레임 length : 8884


,tmax,tmin,depart,dewpoint,heat,cool,snowfall,preciptotal,resultspeed,resultdir,avgspeed
20507,75.0,50.0,-2.0,53.0,2.0,0.0,0.0,0.00,5.3,30.0,6.1
20510,60.0,38.0,-7.0,31.0,16.0,0.0,0.0,0.00,11.9,1.0,12.5
20511,50.0,32.0,-2.0,34.0,24.0,0.0,0.0,0.00,0.3,3.0,0.6
20514,67.0,49.0,-4.0,40.0,7.0,0.0,0.0,0.00,10.6,36.0,11.2
20515,45.0,33.0,-6.0,24.0,26.0,0.0,0.1,0.02,20.4,34.0,20.9


In [13]:
vif_2 = pd.DataFrame()
vif_2["VIF Factor"] = [variance_inflation_factor(df_weather_drop_nonM_2.values, i) for i in range(df_weather_drop_nonM_2.shape[1])]
vif_2["features"] = df_weather_drop_nonM_2.columns
vif_2 = vif_2.sort_values("VIF Factor").reset_index(drop=True)
vif_2

,VIF Factor,features
0,1.106714,snowfall
1,1.273523,preciptotal
2,1.283765,depart
3,2.829128,heat
4,3.851991,cool
5,5.255264,resultdir
6,26.536374,resultspeed
7,39.583314,avgspeed
8,80.527076,tmax
9,100.024915,dewpoint
